[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CERIA-Team/ceria-machinelearning/blob/main/capstone_test_2.ipynb)

In [1]:
!pip install kaggle

import os
os.environ['KAGGLE_USERNAME'] = "pideeell"
os.environ['KAGGLE_KEY'] = "a917e24c299339667d5610765517f26b"

from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [2]:
import pandas as pd
import joblib
import numpy as np

from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

# Music Dataset

## Download and read dataset

In [3]:
!kaggle datasets download -d maharshipandya/-spotify-tracks-dataset

Dataset URL: https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset
License(s): ODbL-1.0
 61% 5.00M/8.17M [00:00<00:00, 33.1MB/s]
100% 8.17M/8.17M [00:00<00:00, 49.6MB/s]


In [4]:
!unzip '/content/-spotify-tracks-dataset.zip' -d '/content/dataset/'
!rm '/content/-spotify-tracks-dataset.zip'

Archive:  /content/-spotify-tracks-dataset.zip
  inflating: /content/dataset/dataset.csv  


In [5]:
df = pd.read_csv('/content/dataset/dataset.csv')
df.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        114000 non-null  int64  
 1   track_id          114000 non-null  object 
 2   artists           113999 non-null  object 
 3   album_name        113999 non-null  object 
 4   track_name        113999 non-null  object 
 5   popularity        114000 non-null  int64  
 6   duration_ms       114000 non-null  int64  
 7   explicit          114000 non-null  bool   
 8   danceability      114000 non-null  float64
 9   energy            114000 non-null  float64
 10  key               114000 non-null  int64  
 11  loudness          114000 non-null  float64
 12  mode              114000 non-null  int64  
 13  speechiness       114000 non-null  float64
 14  acousticness      114000 non-null  float64
 15  instrumentalness  114000 non-null  float64
 16  liveness          11

## Data Cleaning

In [7]:
#select some columns from df
df = df[['track_id','track_name','artists','track_genre','popularity','energy','tempo','valence']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   track_id     114000 non-null  object 
 1   track_name   113999 non-null  object 
 2   artists      113999 non-null  object 
 3   track_genre  114000 non-null  object 
 4   popularity   114000 non-null  int64  
 5   energy       114000 non-null  float64
 6   tempo        114000 non-null  float64
 7   valence      114000 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 7.0+ MB


In [8]:
# print all unique genres
df['track_genre'].unique()

array(['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient',
       'anime', 'black-metal', 'bluegrass', 'blues', 'brazil',
       'breakbeat', 'british', 'cantopop', 'chicago-house', 'children',
       'chill', 'classical', 'club', 'comedy', 'country', 'dance',
       'dancehall', 'death-metal', 'deep-house', 'detroit-techno',
       'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm',
       'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk',
       'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove',
       'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle',
       'heavy-metal', 'hip-hop', 'honky-tonk', 'house', 'idm', 'indian',
       'indie-pop', 'indie', 'industrial', 'iranian', 'j-dance', 'j-idol',
       'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino',
       'malay', 'mandopop', 'metal', 'metalcore', 'minimal-techno', 'mpb',
       'new-age', 'opera', 'pagode', 'party', 'piano', 'pop-film', 'pop',
       'pow

In [9]:
#looking duplicates on track_id
df.duplicated(subset=['track_name']).sum()

40391

In [10]:
#show duplicated value on track_name
df[df.duplicated(subset=['track_name'])]

,track_id,track_name,artists,track_genre,popularity,energy,tempo,valence
18,2qLMf6TuEC3ruGJg4SMMN6,Lucky,Jason Mraz;Colbie Caillat,acoustic,68,0.414,130.088,0.6690
20,3S0OXQeoh0w6AY8WQVckRW,I'm Yours,Jason Mraz,acoustic,75,0.444,150.960,0.7120
22,5TvE3pk05pyFIGdSY9j4DJ,Say Something,A Great Big World;Christina Aguilera,acoustic,70,0.147,141.284,0.0765
28,5QAMZTM5cmLg3fHX9ZbTZi,Winter Wonderland,Jason Mraz,acoustic,0,0.309,145.363,0.6640
29,2qESE1ZeWly7I3YjyTXmXh,Winter Wonderland,Jason Mraz,acoustic,0,0.309,145.363,0.6640
...,...,...,...,...,...,...,...,...
113974,0VVvjjtKyOeT2AEMTSQ33t,Faithful,Chris Tomlin,world-music,34,0.419,131.973,0.3730
113980,64BdW5aHuB3C0QP3qjBJVi,Amen,I AM THEY,world-music,41,0.634,76.994,0.0921
113987,4jDhzTYkEG5GloIWwVeVkc,All My Life - Live,Planetshakers,world-music,40,0.938,128.002,0.4530
113991,0CE0Y6GM75cbrqao8EOAlW,At The Cross (Love Ran Red),Chris Tomlin,world-music,32,0.531,146.003,0.1530


In [11]:
#delete duplicates
df_clean = df.drop_duplicates(subset=['track_name'])
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73609 entries, 0 to 113999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   track_id     73609 non-null  object 
 1   track_name   73608 non-null  object 
 2   artists      73608 non-null  object 
 3   track_genre  73609 non-null  object 
 4   popularity   73609 non-null  int64  
 5   energy       73609 non-null  float64
 6   tempo        73609 non-null  float64
 7   valence      73609 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 5.1+ MB


In [12]:
df_clean.to_csv('music_dataset.csv', index=False)

In [13]:
unique_genres = df_clean['track_genre'].unique()
for genre in unique_genres:
    print(genre)

acoustic
afrobeat
alt-rock
alternative
ambient
anime
black-metal
bluegrass
blues
brazil
breakbeat
british
cantopop
chicago-house
children
chill
classical
club
comedy
country
dance
dancehall
death-metal
deep-house
detroit-techno
disco
disney
drum-and-bass
dub
dubstep
edm
electro
electronic
emo
folk
forro
french
funk
garage
german
gospel
goth
grindcore
groove
grunge
guitar
happy
hard-rock
hardcore
hardstyle
heavy-metal
hip-hop
honky-tonk
house
idm
indian
indie-pop
indie
industrial
iranian
j-dance
j-idol
j-pop
j-rock
jazz
k-pop
kids
latin
latino
malay
mandopop
metal
metalcore
minimal-techno
mpb
new-age
opera
pagode
party
piano
pop-film
pop
power-pop
progressive-house
psych-rock
punk-rock
punk
r-n-b
reggae
reggaeton
rock-n-roll
rock
rockabilly
romance
sad
salsa
samba
sertanejo
show-tunes
singer-songwriter
ska
sleep
soul
spanish
study
swedish
synth-pop
tango
techno
trance
trip-hop
turkish
world-music


## Mapping Songs to Certain Intensitys

In [14]:
from math import e
import pandas as pd

# Create a dictionary to map genres to their respective heart rate intensities
genre_to_intensity = {
    # 'acoustic': {0, 1},
    'afrobeat': {1},
    'alt-rock': {2},
    'alternative': {1},
    # 'ambient': {0},
    'anime': {1, 2},
    'black-metal': {3},
    'bluegrass': {0},
    'blues': {0},
    'brazil': {1},
    'breakbeat': {3},
    'british': {1, 2}, # Broad genre, adjust as needed
    'cantopop': {1},
    'chicago-house': {2},
    # 'children': {0},
    # 'chill': {0},
    'classical': {0},
    'club': {2, 3}, # Broad, depends on specific subgenres
    'comedy': {0, 1},
    'country': {1},
    'dance': {1, 2},
    'dancehall': {2},
    'death-metal': {3},
    'deep-house': {2},
    'detroit-techno': {2, 3},
    'disco': {2},
    'disney': {1},
    'drum-and-bass': {2, 3},
    'dub': {1, 2},
    'dubstep': {2, 3},
    'edm': {2, 3},
    'electro': {2},
    'electronic': {2, 3},
    'emo': {1, 2},
    'folk': {0, 1},
    'forro': {2},
    'french': {1, 2}, # Broad genre, adjust as needed
    'funk': {1},
    'garage': {1, 2},
    'german': {1, 2}, # Broad genre, adjust as needed
    'gospel': {1},
    'goth': {3},
    'grindcore': {3},
    'groove': {1},
    'grunge': {2},
    'guitar': {0, 1, 2, 3}, # Extremely broad, adjust as needed
    'happy': {1},
    'hard-rock': {2, 3},
    'hardcore': {3},
    'hardstyle': {3},
    'heavy-metal': {3},
    'hip-hop': {1, 2, 3},
    'honky-tonk': {1},
    'house': {2, 3},
    'idm': {2},
    'indian': {1, 2}, # Broad genre, adjust as needed
    'indie-pop': {1},
    'indie': {1, 2},
    'industrial': {2, 3},
    'iranian': {1, 2}, # Broad genre, adjust as needed
    'j-dance': {2},
    'j-idol': {1, 2},
    'j-pop': {1},
    'j-rock': {2},
    'jazz': {0, 1},
    'k-pop': {1},
    # 'kids': {0},
    'latin': {1, 2},
    'latino': {1, 2},
    'malay': {1},
    'mandopop': {1},
    'metal': {2, 3},
    'metalcore': {3},
    'minimal-techno': {2},
    'mpb': {1},
    'new-age': {0},
    'opera': {0, 1}, # Can vary greatly in intensity
    'pagode': {2},
    'party': {2, 3},
    'piano': {0, 1},
    'pop-film': {1},
    'pop': {1, 2},
    'power-pop': {1},
    'progressive-house': {2},
    'psych-rock': {1, 2},
    'punk-rock': {2},
    'punk': {3},
    'r-n-b': {1, 2},
    'reggae': {1, 2},
    'reggaeton': {2},
    'rock-n-roll': {1, 2},
    'rock': {2, 3},
    'rockabilly': {1},
    'romance': {0, 1},
    'sad': {0, 1},
    'salsa': {2},
    'samba': {2},
    'sertanejo': {1, 2},
    'show-tunes': {1},
    'singer-songwriter': {0, 1},
    'ska': {2},
    # 'sleep': {0},
    'songwriter': {0},
    'soul': {1},
    'spanish': {1, 2}, # Broad genre, adjust as needed
    'study': {0},
    'swedish': {1, 2}, # Broad genre, adjust as needed
    'synth-pop': {1},
    'tango': {1, 2},
    'techno': {2, 3},
    'trance': {2, 3},
    'trip-hop': {1, 2},
    'turkish': {1, 2}, # Broad genre, adjust as needed
    'world-music': {0, 1, 2}
}

In [15]:
# Counters for assigned and not assigned intensities
assigned_count = 0
not_assigned_count = 0

def assign_intensity(row):
    global assigned_count, not_assigned_count
    genre = row['track_genre'].lower()
    tempo = row['tempo']
    energy = row['energy']

    intensity_set = genre_to_intensity.get(genre)
    if intensity_set is None:
        print(f"Warning: Genre not found - '{row['track_name']}' - Genre: '{genre}'")
        not_assigned_count += 1
        return None

    # Find matching intensities based on tempo
    matched_intensities = set()
    if 0 in intensity_set and 50 <=tempo <= 90 and energy >= 0.7:
        matched_intensities.add(0)
    if 1 in intensity_set and 80 <= tempo <= 120 and energy >= 0.75:
        matched_intensities.add(1)
    if 2 in intensity_set and 110 <= tempo <= 140 and energy >= 0.85:
        matched_intensities.add(2)
    if 3 in intensity_set and tempo > 130 and energy >= 0.9:
        matched_intensities.add(3)

    if matched_intensities:
        assigned_count += 1
        return matched_intensities
    else:
        print(f"Warning: Intensity not assigned - '{row['track_name']}' - Genre: '{genre}' - Tempo: {tempo} - Energy: {energy}")
        not_assigned_count += 1
        return None

df_assign = df_clean.copy()  # Ensure df_clean is available
# Apply the function to each row to assign intensity
df_assign['intensity'] = df_clean.apply(assign_intensity, axis=1)

# # Drop rows where intensity could not be determined
# df_assign = df_assign.dropna(subset=['intensity'])

# Display the first few rows of the dataframe to verify the result
print(f'\nNumber of assigned intensities: {assigned_count}')
print(f'Number of skipped rows: {not_assigned_count}')
print(df_assign.head(30))



Streaming output truncated to the last 5000 lines.

Number of assigned intensities: 11763
Number of skipped rows: 61846
                  track_id                                   track_name  \
0   5SuOikwiRyPMVoIQDJUgSV                                       Comedy   
1   4qPNDBW1i3p13qLCt0Ki3A                             Ghost - Acoustic   
2   1iJBSr7s7jYXzM8EGcbK5b                               To Begin Again   
3   6lfxq3CG4xtTiEg7opyCyx                   Can't Help Falling In Love   
4   5vjLSffimiIP26QG5WcN2K                                      Hold On   
5   01MVOl9KtVTNfFiBU9I7dc                         Days I Will Remember   
6   6Vc5wAMmXdKIAM7WUoEb7N                                Say Something   
7   1EzrEOXmMH3G43AXT1y7pA                                    I'm Yours   
8   0IktbUcnAGrvD03AWnz3Q8                                        Lucky   
9   7k9GuJYLp2AzqokyEdwEw2                                       Hunger   
10  4mzP5mHkRvGxdhdGdAH7EJ                         Give

In [16]:
# drop df_assign if intensity = None
df_assign = df_assign.dropna(subset=['intensity'])

# df_assign.head(50)
df_assign.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11763 entries, 1001 to 113992
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   track_id     11763 non-null  object 
 1   track_name   11763 non-null  object 
 2   artists      11763 non-null  object 
 3   track_genre  11763 non-null  object 
 4   popularity   11763 non-null  int64  
 5   energy       11763 non-null  float64
 6   tempo        11763 non-null  float64
 7   valence      11763 non-null  float64
 8   intensity    11763 non-null  object 
dtypes: float64(3), int64(1), object(5)
memory usage: 919.0+ KB


In [17]:
df_assign['intensity'].value_counts()

intensity
{1}       3817
{2}       3666
{3}       3158
{2, 3}     474
{1, 2}     269
{0}        249
{0, 1}     130
Name: count, dtype: int64

In [18]:
# Expand the intensity set to individual rows for proper counting
df_assign_expanded = df_assign.explode('intensity')

# Group by 'track_genre' and 'intensity' and count the number of songs
genre_intensity_counts = df_assign_expanded.groupby(['track_genre', 'intensity']).size().reset_index(name='count')

# Display the counts
print(genre_intensity_counts)

# genre_intensity_counts.to_csv('genre_intensity_counts.csv', index=False)

     track_genre  intensity  count
0       afrobeat          1    232
1       alt-rock          2     87
2    alternative          1     44
3          anime          1    175
4          anime          2     89
..           ...        ...    ...
153      turkish          1     82
154      turkish          2     25
155  world-music          0     26
156  world-music          1     65
157  world-music          2     55

[158 rows x 3 columns]


In [19]:
genre_intensity_counts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   track_genre  158 non-null    object
 1   intensity    158 non-null    int64 
 2   count        158 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 3.8+ KB


# Fitness Dataset

## Download and read dataset

In [20]:
# download dataset
!kaggle datasets download -d arashnic/fitbit
!unzip '/content/fitbit.zip' -d '/content/dataset/'
!rm '/content/fitbit.zip'

Dataset URL: https://www.kaggle.com/datasets/arashnic/fitbit
License(s): CC0-1.0
 90% 39.0M/43.3M [00:00<00:00, 51.9MB/s]
100% 43.3M/43.3M [00:01<00:00, 43.5MB/s]
Archive:  /content/fitbit.zip
  inflating: /content/dataset/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/dailyActivity_merged.csv  
  inflating: /content/dataset/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/heartrate_seconds_merged.csv  
  inflating: /content/dataset/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/hourlyCalories_merged.csv  
  inflating: /content/dataset/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/hourlyIntensities_merged.csv  
  inflating: /content/dataset/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/hourlySteps_merged.csv  
  inflating: /content/dataset/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/minuteCaloriesNarrow_merged.csv  
  inflating: /content/dataset/mturkfitbit_export_3.12.16-4.11

In [21]:
# Read CSV
df_Intensities = pd.read_csv('/content/dataset/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/minuteIntensitiesNarrow_merged.csv')
df_HR = pd.read_csv('/content/dataset/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/heartrate_seconds_merged.csv')
df_Calories = pd.read_csv('/content/dataset/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/minuteCaloriesNarrow_merged.csv')
df_METs = pd.read_csv('/content/dataset/mturkfitbit_export_3.12.16-4.11.16/Fitabase Data 3.12.16-4.11.16/minuteMETsNarrow_merged.csv')

df_Intensities1 = pd.read_csv('/content/dataset/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/minuteIntensitiesNarrow_merged.csv')
df_HR1 = pd.read_csv('/content/dataset/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/heartrate_seconds_merged.csv')
df_Calories1 = pd.read_csv('/content/dataset/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/minuteCaloriesNarrow_merged.csv')
df_METs1 = pd.read_csv('/content/dataset/mturkfitbit_export_4.12.16-5.12.16/Fitabase Data 4.12.16-5.12.16/minuteMETsNarrow_merged.csv')

In [22]:
# Concatenate df
df_HR = pd.concat([df_HR, df_HR1], ignore_index=True)
df_Intensities = pd.concat([df_Intensities, df_Intensities1], ignore_index=True)
df_Calories = pd.concat([df_Calories, df_Calories1], ignore_index=True)
df_METs = pd.concat([df_METs, df_METs1], ignore_index=True)

In [23]:
df_Intensities.info()
df_HR.info()
df_Calories.info()
df_METs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2770620 entries, 0 to 2770619
Data columns (total 3 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   Id              int64 
 1   ActivityMinute  object
 2   Intensity       int64 
dtypes: int64(2), object(1)
memory usage: 63.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3638339 entries, 0 to 3638338
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Id      int64 
 1   Time    object
 2   Value   int64 
dtypes: int64(2), object(1)
memory usage: 83.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2770620 entries, 0 to 2770619
Data columns (total 3 columns):
 #   Column          Dtype  
---  ------          -----  
 0   Id              int64  
 1   ActivityMinute  object 
 2   Calories        float64
dtypes: float64(1), int64(1), object(1)
memory usage: 63.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2770620 entries, 0 to 2770619
Data columns (total 3 columns):
 #

In [24]:
# Rename columns
df_HR.rename(columns={'Time': 'ActivityMinute'}, inplace=True)
df_HR.rename(columns={'Value': 'HR'}, inplace=True)

# Merge df
df_all = df_Intensities.merge(df_HR, on=['Id', 'ActivityMinute'], how='inner') \
                       .merge(df_Calories, on=['Id', 'ActivityMinute'], how='inner') \
                       .merge(df_METs, on=['Id', 'ActivityMinute'], how='inner')

df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303199 entries, 0 to 303198
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Id              303199 non-null  int64  
 1   ActivityMinute  303199 non-null  object 
 2   Intensity       303199 non-null  int64  
 3   HR              303199 non-null  int64  
 4   Calories        303199 non-null  float64
 5   METs            303199 non-null  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 13.9+ MB


In [25]:
df_all['Intensity'].value_counts()

Intensity
0    221728
1     67067
3      9140
2      5264
Name: count, dtype: int64

## Data Cleaning

In [26]:
# Balance the dataset

# feature, target column
X = df_all.drop(columns=['Intensity'])
y = df_all['Intensity']

# undersampling
under_sampling = RandomUnderSampler(random_state=42)
X_us, y_us = under_sampling.fit_resample(X, y)
df_all_resampled = pd.concat([X_us, y_us], axis=1)

# new distribution
print(df_all_resampled['Intensity'].value_counts())
df_all_resampled.info()

Intensity
0    5264
1    5264
2    5264
3    5264
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21056 entries, 0 to 21055
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              21056 non-null  int64  
 1   ActivityMinute  21056 non-null  object 
 2   HR              21056 non-null  int64  
 3   Calories        21056 non-null  float64
 4   METs            21056 non-null  int64  
 5   Intensity       21056 non-null  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 987.1+ KB


In [27]:
df_all_resampled['METs'].mean()

43.19813829787234

In [28]:
df_all_resampled.groupby('Intensity').agg({
    'HR': 'mean',
    'Calories': 'mean',
    'METs': 'mean'
})

,HR,Calories,METs
Intensity,,,
0,67.973214,1.150020,10.715616
1,86.471315,3.599079,33.902546
2,101.630319,5.648026,51.873290
3,119.462576,8.644752,76.301102


# Mapping Heartrate intensity to Music List

In [29]:
# Separate features and target
X = df_all_resampled[['HR']]#, 'Calories', 'METs']]
y = df_all_resampled['Intensity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Random Forest Classifier

In [30]:
# Best parameters from Grid Search
# Best parameters found:  {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 5}

# Best parameters
best_params = {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 10, 'random_state': 42}

# train final model
final_model = RandomForestClassifier(**best_params)
final_model.fit(X_train, y_train)

y_pred = final_model.predict(X_test)

# Eval
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Classification Report: \n", classification_report(y_test, y_pred))

Accuracy:  0.6132478632478633
Classification Report: 
               precision    recall  f1-score   support

           0       0.76      0.76      0.76      1049
           1       0.50      0.47      0.48      1102
           2       0.48      0.52      0.50      1015
           3       0.72      0.71      0.72      1046

    accuracy                           0.61      4212
   macro avg       0.61      0.61      0.61      4212
weighted avg       0.61      0.61      0.61      4212



In [31]:
# Feature importance
feature_importances = final_model.feature_importances_
features = ['HR']
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print(importance_df)

  Feature  Importance
0      HR         1.0


In [32]:
df_all_resampled['HR']

0         73
1         79
2         77
3         61
4         56
        ... 
21051    112
21052    137
21053    108
21054    111
21055     99
Name: HR, Length: 21056, dtype: int64

In [33]:
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(final_model, 'model.pkl')


# Save the model and the scaler
with open('final_model.bin', 'wb') as model_file:
    joblib.dump(final_model, model_file)

with open('scaler.bin', 'wb') as scaler_file:
    joblib.dump(scaler, scaler_file)

# Save DataFrame
df_assign.to_csv('filtered_song.csv', index=False)


In [34]:
import json
from sklearn.ensemble import RandomForestClassifier

# save final_model to json
model_params = final_model.get_params()

with open('model.json', 'w') as json_file:
    json.dump(model_params, json_file)

# Inference

In [35]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

# Load the DataFrame from the saved CSV file
df_assign = pd.read_csv('filtered_song.csv')

# Ensure that 'intensity' is converted to sets
df_assign['intensity'] = df_assign['intensity'].apply(lambda x: eval(x) if isinstance(x, str) else x)
df_assign['intensity'] = df_assign['intensity'].apply(lambda x: set(x) if isinstance(x, (list, np.ndarray)) else x)

# Load the model and scaler
with open('final_model.bin', 'rb') as model_file:
    model = joblib.load(model_file)

with open('scaler.bin', 'rb') as scaler_file:
    scaler = joblib.load(scaler_file)


def recommend_songs(predicted_intensity, df_assign, num_songs=5):
    # Filter songs matching predicted intensity
    recommended_df = df_assign[df_assign['intensity'].apply(lambda x: predicted_intensity in x)].copy()

    # If no songs match the exact intensity, expand to adjacent intensities
    if recommended_df.empty:
        if predicted_intensity == 0:
            recommended_df = df_assign[df_assign['intensity'].apply(lambda x: 1 in x)].copy()
        elif predicted_intensity == 3:
            recommended_df = df_assign[df_assign['intensity'].apply(lambda x: 2 in x)].copy()
        else:
            recommended_df = df_assign[df_assign['intensity'].apply(lambda x: (predicted_intensity - 1 in x) or (predicted_intensity + 1 in x))].copy()

    # Apply popularity-based weighting for random selection
    recommended_df['weight'] = recommended_df['popularity'].apply(
        lambda pop: 1 if pop > 60 else (0.2 if 45 < pop <= 60 else 0.01)
    )

    # If not enough songs after filtering and weighting, recommend whatever is available
    if len(recommended_df) < num_songs:
        num_songs = len(recommended_df)

    # Randomly select songs based on weighted probability
    recommended_songs = recommended_df.sample(n=num_songs, weights='weight')

    return recommended_songs[['track_id','track_name', 'artists', 'track_genre', 'intensity', 'energy','popularity']]

# Example usage
Hr = 120

# Create DataFrame for prediction
new_data = pd.DataFrame({
    'HR': [Hr],
})

# Scale input and predict intensity
new_data_scaled = scaler.transform(new_data)
predicted_intensity = model.predict(new_data_scaled)[0]

# Get song recommendations
recommendations = recommend_songs(predicted_intensity, df_assign, 1)
print(recommendations)


                    track_id track_name         artists track_genre intensity  \
1675  5pqaZzkZ5AnjZs5AV3Bnvn    INERTIA  F.O.O.L;Caster        club       {3}   

      energy  popularity  
1675   0.918          40  


In [36]:
recommendations['track_id']

1675    5pqaZzkZ5AnjZs5AV3Bnvn
Name: track_id, dtype: object

In [32]:
from flask import Flask, request, jsonify
import pickle
import pandas as pd
import joblib

app = Flask(__name__)

# Load pre-trained model and other necessary files
with open('Model/scaler.bin', 'rb') as f:
  scaler = joblib.load(f)

with open('Model/final_model.bin', 'rb') as e:
  model = joblib.load(e)

# Assume df_assign is preloaded from a CSV file
df_assign = pd.read_csv('Datasets/filtered_song.csv')

def recommend_songs(predicted_intensity, df_assign, num_songs=5):
    recommended_df = df_assign[df_assign['intensity'].apply(lambda x: predicted_intensity in x)].copy()

    if recommended_df.empty:
        if predicted_intensity == 0:
            recommended_df = df_assign[df_assign['intensity'].apply(lambda x: 1 in x)].copy()
        elif predicted_intensity == 3:
            recommended_df = df_assign[df_assign['intensity'].apply(lambda x: 2 in x)].copy()
        else:
            recommended_df = df_assign[df_assign['intensity'].apply(lambda x: (predicted_intensity - 1 in x) or (predicted_intensity + 1 in x))].copy()

    recommended_df['weight'] = recommended_df['popularity'].apply(
        lambda pop: 1 if pop > 60 else (0.2 if 45 < pop <= 60 else 0.01)
    )

    if len(recommended_df) < num_songs:
        num_songs = len(recommended_df)

    recommended_songs = recommended_df.sample(n=num_songs, weights='weight')

    return recommended_songs[['track_id', 'track_name', 'artists', 'track_genre', 'intensity', 'energy', 'popularity']]

@app.route('/recommend', methods=['POST'])
def recommend():
    user_data = request.json
    hr = user_data['HR']

    # Create DataFrame for prediction
    new_data = pd.DataFrame({
        'HR': [hr],
    })

    # Scale input and predict intensity
    new_data_scaled = scaler.transform(new_data)
    predicted_intensity = model.predict(new_data_scaled)[0]

    # Get song recommendations
    recommendations = recommend_songs(predicted_intensity, df_assign, 5)

    # Format the recommendations as JSON
    response = recommendations.to_dict(orient='records')

    return jsonify(response)

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0')


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.62.184:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\doubl\PycharmProjects\Simulasi Ujian TensorFlow Developer Certificate\.venv\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [16]:
import os
if os.path.exists('Model\model.pkl'):
    print("File exists.")
else:
    print("File does not exist.")


File exists.


In [23]:
import sklearn

sklearn.__version__

'1.3.2'